In [13]:
%matplotlib inline

import datetime as dt
import itertools as it

from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import synapseclient
from synapseclient import Activity, Schema, Table, as_table_columns

InteractiveShell.ast_node_interactivity = 'all'
syn = synapseclient.Synapse()
syn.login()

Welcome, Luke Waninger!



In [14]:
v2sid = 'syn17037408'
v2r = pd.read_csv(syn.get(v2sid).path, parse_dates=['date']).rename(columns={
    'username':'participant_id',
    'date':'dt_passive'
})

leading = ['participant_id', 'dt_passive']
v2 = v2r.reindex(labels=
    leading + sorted(list(set(v2r.columns)-set(leading))),
    axis=1                  
)

v2.location_variance = np.round(v2.location_variance, 3)

v2.head()

,participant_id,dt_passive,came_to_work,distance_active,distance_high_speed_transportation,distance_powered_vehicle,distance_walking,hours_accounted_for,hours_active,hours_at_home,...,hours_high_speed_transportation,hours_of_sleep,hours_powered_vehicle,hours_spent_in_top_3_clusters,hours_stationary,hours_stationary_non_home_work,hours_walking,hours_without_data,location_variance,number_of_clusters
0,EN00222,2016-09-12,True,5978.0,0.0,16807.7,8567.0,8,0.517,0.000,...,0.0,NaN,0.517,0.0,9.350,5.500,2.933,16,-2.788,1
1,EN00222,2016-09-13,False,12907.0,0.0,45988.3,2589.0,12,1.167,0.000,...,0.0,0.150,1.050,0.0,21.083,17.017,0.683,12,-2.619,0
2,EN00222,2016-09-14,False,4655.0,0.0,11578.1,0.0,5,0.417,1.367,...,0.0,0.150,0.350,0.0,16.200,13.350,0.000,19,-2.768,1
3,EN00222,2016-09-15,False,10569.0,0.0,16984.7,587.0,6,0.933,0.000,...,0.0,17.167,0.467,0.0,18.483,17.167,0.167,18,-2.740,0
4,EN00222,2016-09-16,False,9962.0,0.0,13008.3,582.0,5,0.733,0.000,...,0.0,NaN,0.367,0.0,22.583,18.400,0.167,19,-3.121,0


In [19]:
cheat = []
for c in v2.columns:
    cheat.append(
        (c, str(v2[c].dtype).replace('object', 'str').replace('float64', 'float').replace('int64', 'int'))
    )
pd.DataFrame(cheat, columns=['Variable Name', 'Data Type'])

,Variable Name,Data Type
0,participant_id,str
1,dt_passive,datetime64[ns]
2,came_to_work,bool
3,distance_active,float
4,distance_high_speed_transportation,float
5,distance_powered_vehicle,float
6,distance_walking,float
7,hours_accounted_for,int
8,hours_active,float
9,hours_at_home,float


In [15]:
final = syn.store(Table(
    Schema(
            name='GPS Features (v2)',
            columns=as_table_columns(v2), 
            parent='syn10848316'),
        v2
    )
)

In [16]:
final = syn.setProvenance(
    'syn17060502',
    activity=Activity(
        name='Generate Public GPS (v2) Table Data',
        description='Process the data collected during study ',
        used=[v2sid],
        executed=[
            dict(
                name='IPython Notebook',
                url='https://github.com/apratap/BRIGHTEN-Data-Release/blob/master/Create_V2GpsFeatures_datafiles.ipynb'
            )
        ]
    )
)